In [ ]:
import ast
import sys

'''
FILENAME = sys.argv[1]
min_idx = int( sys.argv[2] )
max_idx = int( sys.argv[3] )
RESULTS = ast.literal_eval( sys.argv[4] )
PL_CURVES = ast.literal_eval( sys.argv[5] )
AL_CURVES = ast.literal_eval( sys.argv[6] )
SSL_CURVES = ast.literal_eval( sys.argv[7] )
MODELS = sys.argv[8].split(',')
#thr_conf = float(sys.argv[9])
'''

FILENAME = 'OBESIDAD'
min_idx = 1
max_idx = 5
RESULTS = False
PL_CURVES = True
AL_CURVES = False
SSL_CURVES = False
#results
MODELS = [
        'cregex*random'
        #'cregex*rf-n1.svm-n1.nb-n1.sw.random'
       ]
#learning curves
#MODELS = ['cregex*rf-n1', 'cregex*svm-n1','cregex*nb-n1', 'cregex*random']
#MODELS = ['cregex*random']

sys.path.append('../FREGEX')

import warnings
warnings.filterwarnings("ignore")
import logging
logging.captureWarnings(True)
logging.disable(sys.maxsize)
from utils import *
from curves import Curves
SEED = 42
seed_everything()

PRED_TYPE = 'error'
N_CLASSES = {'FUMADOR':2, 'OBESIDAD':2, 'OBESIDAD_TIPOS':3}[FILENAME]
HYPERPARAMS['bert']['n_classes'] = N_CLASSES
HYPERPARAMS['cregex-bert']['n_classes'] = N_CLASSES
create_paths(FILENAME)
    
with open( os.path.join( os.getcwd(), 'snippets_procesados_'+FILENAME),  'rb') as a:
    data = pickle.load(a)    
    data = sorted(data, key = lambda x:x[0], reverse = False)
    DATA = np.array( [snippet for snippet, classe in data] )#[:600]
    CLASSES = np.array( [classe for snippet, classe in data])#[:600]

print(FILENAME)
RUNS = 1
FOLDS = 5
folds = KFold(n_splits = FOLDS, shuffle = False, random_state = SEED)
idxs = np.arange(0, len(DATA))    
    
for r in range(RUNS):
    idxs = shuffle(idxs, random_state = SEED)
    CLASSES = CLASSES[idxs]
    DATA = DATA[idxs]    
    k = -1
    for train_index, test_index in folds.split(idxs):
        k+=1        
        print( '\nrun:', r+1, 'fold: ', k+1 )
        print('AVG-train:', np.mean(CLASSES[train_index]), 'AVG-test:', np.mean(CLASSES[test_index]))
        if (k+1) not in list(range(min_idx, max_idx+1)):
            continue
        X_train = copy.deepcopy( DATA[train_index] )
        y_train = copy.deepcopy( CLASSES[train_index] )
        X_test = copy.deepcopy( DATA[test_index] )
        y_test = copy.deepcopy( CLASSES[test_index] )   

        for MODEL in MODELS:
            print(MODEL)
            MODEL_aux = MODEL.replace('*','-')

            if RESULTS:
                CURVE = 'RESULTS'
                print(CURVE)
                curve = Curves(X_train, y_train,
                               X_test, y_test, 
                               N_CLASSES, CURVE, MODEL, FILENAME)
                pred, model = curve.model_selection(X_train, y_train, X_test, FILENAME, True, True, False)
                save = []
                if 'cregex' in MODEL:    
                    tokens = copy.deepcopy(model.tokens)
                    for MODEL_NAME in pred:
                        prob_path = ''
                        if 'cregex' in MODEL_NAME: 
                            prob_path = '_'+str(THR_CONF)
                        print('%s-acc: %.2f' %(MODEL_NAME, 100*accuracy_score(y_test, pred[MODEL_NAME])) ) 
                        save = [y_test, pred[MODEL_NAME], model.HYPERPARAMS[MODEL_NAME], tokens[MODEL_NAME], model.distribution]
                        with open( os.path.join( os.getcwd(), 'out', 'RESULTS', FILENAME, FILENAME+'_results_'+MODEL_NAME+prob_path+'_r'+str(r+1)+'_f'+str(k+1)+'.pkl' ), 'wb') as a:
                            pickle.dump(save, a, protocol = 2)
                else:
                    tokens = copy.deepcopy(curve.tokens)
                    print('%s-acc: %.2f' %(MODEL, 100*accuracy_score(y_test, pred)) )  
                    save = [y_test, pred, curve.HYPERPARAMS, tokens, None]
                    with open( os.path.join( os.getcwd(), 'out', 'RESULTS', FILENAME, FILENAME+'_results_'+MODEL+'_r'+str(r+1)+'_f'+str(k+1)+'.pkl' ), 'wb') as a:
                        pickle.dump(save, a, protocol = 2)
                        
                del curve
                del pred
                gc.collect()

            if PL_CURVES:
                CURVE = 'PL'
                print(CURVE)
                curve = Curves(X_train, y_train,
                               X_test, y_test, 
                               N_CLASSES, CURVE, MODEL, FILENAME, PRED_TYPE)              
                curve.learningCurve() 
                results_lc_pl = defaultdict(list)
                if PRED_TYPE == 'error':
                    results_lc_pl[CURVE].append( [ [], curve.results['x'], curve.results['y']] )

                else:
                    results_lc_pl[CURVE].append( [y_test, curve.results['x'], curve.results['y']] )
                    
                results_lc_pl['params_'+MODEL+'_'+CURVE].append(curve.HYPERPARAMS)
                with open( os.path.join( os.getcwd(), 'out', 'RESULTSLC', CURVE, FILENAME, FILENAME+'_results_lc_pl_'+MODEL_aux+'_r'+str(r+1)+'_f'+str(k+1)+'.pkl' ), 'wb') as a:
                    pickle.dump(results_lc_pl, a, protocol = 2)
                del curve
                del results_lc_pl
                gc.collect()

            if AL_CURVES:
                CURVE = 'AL'
                print(CURVE)

                curve = Curves(X_train, y_train, 
                               X_test, y_test, 
                               N_CLASSES, CURVE, MODEL, FILENAME)              
                curve.learningCurve()
                results_lc_al = defaultdict(list)
                results_lc_al[CURVE].append( [y_test, curve.results['x'], curve.results['y']] )
                results_lc_al['scores_'+CURVE].append( [curve.results['x'], curve.results['scores']  ] )
                results_lc_al['params_'+MODEL+'_'+CURVE].append(curve.HYPERPARAMS)
                with open( os.path.join( os.getcwd(), 'out', 'RESULTSLC', CURVE, FILENAME, FILENAME+'_results_lc_al_'+MODEL_aux+'_r'+str(r+1)+'_f'+str(k+1)+'.pkl' ), 'wb') as a:
                    pickle.dump(results_lc_al, a, protocol = 2)
                del curve
                del results_lc_al
                gc.collect()

            if SSL_CURVES:
                CURVE = 'SSLAL'
                print(CURVE)

                curve = Curves(X_train, y_train, 
                               X_test, y_test, 
                               N_CLASSES, CURVE, MODEL, FILENAME)              
                curve.learningCurve()  
                results_lc_sslal = defaultdict(list)
                results_lc_sslal[CURVE].append( [y_test, curve.results['x'], curve.results['y']] )
                results_lc_sslal['samples_'+CURVE].append( [ curve.results['x'], curve.results['samples'] ] )
                results_lc_sslal['distribution_'+CURVE].append( [ curve.results['x'], curve.results['distribution'] ] )
                results_lc_sslal['scores_'+CURVE].append( [curve.results['x'], curve.results['scores']  ] )
                results_lc_sslal['params_'+MODEL+'_'+CURVE].append( curve.HYPERPARAMS )
                with open( os.path.join( os.getcwd(), 'out', 'RESULTSLC', CURVE, FILENAME, FILENAME+'_results_lc_sslal_'+MODEL_aux+'_r'+str(r+1)+'_f'+str(k+1)+'.pkl' ), 'wb') as a:
                    pickle.dump(results_lc_sslal, a, protocol = 2)
                del curve
                del results_lc_sslal
                gc.collect()
            
        del X_train
        del X_test
        del y_train
        del y_test
        gc.collect()
